# metacognitive prompting

### import statements

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv
import sys
import queue

load_dotenv()

In [ ]:
sys.path.append(os.getenv("SYS_PATH"))
print(f'{os.getenv("SYS_PATH")}\n\n')
print(sys.path)

In [ ]:
from openai import OpenAI
from ipywidgets import widgets
from IPython.display import display, clear_output
from bson.objectid import ObjectId
from typing import Optional
import threading

from models.user import create_user
from models.chat_message import create_chat_message
from models.conversation import create_conversation
from models.chatbot_response import create_chatbot_response
from models.expectation import create_expectation
from models.expectation_revision import create_expectation_revision
from models.violation import create_violation

from utils.model_operations import create_model, get_model
from services.mock_service import mock_user_data, mock_expectation_data, mock_expectation_revision_data, mock_message_data, mock_violation_data, mock_chatbot_response_data
from utils.helpers import CustomOutput

In [ ]:
%run ../services/mongo_service.py

In [ ]:
from services.mongo_service import mongo_client, ping_client
db_client = mongo_client()
ping_client(db_client)

In [ ]:
text_input = ""
chat_output = CustomOutput()

user_input_queue = queue.Queue()
set_user = None
set_conversation = None
active = True

processing_message = threading.Event()
button_click_event = threading.Event()

In [ ]:
test_user = create_user(email=mock_user_data["email"], password=mock_user_data["password"])
print(f"test_user: {test_user}\nis of type{type(test_user)}")
saved_user_id = create_model(collection_name="users", model_data=test_user, client=db_client)

In [ ]:
test_conversation = create_conversation(user_id=saved_user_id)
saved_conversation_id = create_model(collection_name="conversations", model_data=test_conversation, client=db_client)

In [ ]:
test_expectation = create_expectation(reasoning=mock_expectation_data["reasoning"], user_predictions=mock_expectation_data["user_predictions"], additional_data=mock_expectation_data["additional_data"])
saved_expectation_id = create_model(collection_name="expectations", model_data=test_expectation, client=db_client)

In [ ]:
test_expectation_revision = create_expectation_revision(revised_input_possibilities=mock_expectation_revision_data["revised_input_possibilities"], prediction_error=-0.045, initial_expectation_id=saved_expectation_id)
saved_expectation_revision_id = create_model(collection_name="expectation_revisions", model_data=test_expectation_revision, client=db_client)

In [ ]:
test_message = create_chat_message(user_id=saved_user_id, content=mock_message_data["content"], conversation_id=saved_conversation_id)
saved_message_id = create_model(collection_name="chat_messages", model_data=test_message, client=db_client)

In [ ]:
test_violation = create_violation(last_llm_response_id=None, expectation_id=saved_expectation_id, voe_thought=mock_violation_data["voe_thought"])
saved_violation_id = create_model(collection_name="violations", model_data=test_violation, client=db_client)

In [ ]:
get_model(collection_name="users", model_id=saved_user_id, client=db_client)

In [ ]:
print(mock_expectation_data["reasoning"])

In [ ]:
apikey = os.getenv("OPENAI_API_KEY")
openai_client = OpenAI(
    api_key=apikey
)
print(openai_client)

:::{admonition} `stream_chatbot` docs
:class: dropdown 

### `stream_chatbot(message)`

This function interacts with the chatbot using stream functionality.

**Parameters:**
- `message`: The message sent by the user to the chatbot.

**Returns:**
- None

**Functionality:**
- Sends the user message to the chatbot.
- Receives and prints the response from the chatbot.
```

In [ ]:
def stream_chatbot(message):
    chat_completion_stream = openai_client.chat.completions.create(
        messages=[
            {
                "role":"user",
                "content":message,
             },
        ],
        model="gpt-3.5-turbo",
        stream=True
    )
    print(chat_completion_stream)
    for chunk in chat_completion_stream:
        if chunk.choices[0].delta.content is not None:
            chat_output.append_stdout(f"{chunk.choices[0].delta.content}")
    chat_output.append_stdout(f"\n")

:::{admonition} `chatbot` docs
:class: dropdown

### `chatbot(user_message)`

This function interacts with the chatbot based on user input.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Initiates interaction with the chatbot by passing the user's message.
- Prints the chatbots response.


In [ ]:
def chatbot(user_message):
    global active
    processing_message.set()
    chat_output.append_stdout("Chatbot: ")
    if user_message.lower() in ["bye!", "quit", "exit"]:
        chat_output.append_stdout("BYE\n\n")
        active = False
    else:
        stream_chatbot(user_message)
    processing_message.clear()

:::{admonition} `print_user_message` docs
:class: dropdown

### `print_user_message(user_message)`

This function prints the user's message.

**Parameters:**
- `user_message`: The message provided by the user.

**Returns:**
- None

**Functionality:**
- Prints the user's message in the format: "You: [user_message]".


In [ ]:
def print_user_message(user_message):
    chat_output.append_stdout(f"You: {user_message}\n\n")

In [ ]:
def handle_user_message(user_message):
    chatbot(user_message)

In [ ]:
def handle_button_click():
    chat_output.append_stdout(f"handle button clicked\n\n", debug=True)
    chat_output.append_stdout(f"button_click_event.is_set(): {button_click_event.is_set()}\n\n", debug=True)
    chat_output.append_stdout(f"handle button clicked before wait\n\n", debug=True)
    chat_output.append_stdout(f"handle button clicked after wait\n\n", debug=True)
    chat_output.append_stdout(f"std statement, processing msg: {processing_message.is_set()}\n\n", debug=True)
    if not processing_message.is_set():
        chat_output.append_stdout(f"BUTTON CLICK FN TRIGGERED\nprocessing message set status: {processing_message.is_set()}\n\n", debug=True)
        chat_output.append_stdout(f"text_input.value: {text_input.value}\n\n", debug=True)
        user_input_queue.put(text_input.value)
        text_input.value=''
    chat_output.append_stdout(f"handle button clicked after clearing button event and setting queue event\n\n", debug=True)
        

In [ ]:
def initialize_chat(user_id: ObjectId, conversation_id: Optional[ObjectId] = None):
    global set_user, set_conversation
    set_user = get_model(collection_name="users", model_id=saved_user_id, client=db_client)
    if not conversation_id:
        new_conversation = create_conversation(user_id=user_id)
        conversation_id = create_model(collection_name="conversations", model_data=new_conversation, client=db_client)
    set_conversation = get_model(collection_name="conversations", model_id=conversation_id, client=db_client)

In [ ]:
def user_prediction_task() -> dict:
    # invoke reasoning process with Engagement Monitor Service
    reasoning = mock_expectation_data["reasoning"]
    
    # invoke user predictions with LLM Service
    user_predictions = mock_expectation_data["user_predictions"]
    
    # invoke vector db fact fetching with Knowledge Assessment Service
    additional_data = mock_expectation_data["additional_data"]
    
    # create and save Expectation
    new_expectation = create_expectation(reasoning=reasoning, user_predictions=user_predictions, additional_data=additional_data)
    expectation_id = create_model(collection_name="expectations", model_data=new_expectation, client=db_client)
    
    # invoke prediction improvement with LLM Service
    improved_predictions = mock_expectation_revision_data["revised_input_possibilities"]
    
    # invoke prediction error calculation with ??? Service
    prediction_error = 0.0
    
    # create and save ExpectationRevision
    new_expectation_revision = create_expectation_revision(revised_input_possibilities=improved_predictions, prediction_error=prediction_error, initial_expectation_id=expectation_id)
    expectation_revision_id = create_model(collection_name="expectation_revisions", model_data=new_expectation_revision, client=db_client)
    
    expectation_revision_dict = get_model(collection_name="expectation_revisions", model_id=expectation_revision_id, client=db_client)
    
    chat_output.append_stdout(f"USER PREDICTION TASK COMPLETED\nexpectation_revision_dict ID: {expectation_revision_dict['_id']}\n\n", debug=True)
    
    return expectation_revision_dict
    

In [ ]:
def user_input_task() -> dict:
    chat_output.append_stdout(f"USER INPUT TASK\n\n", debug=True)
    chat_output.append_stdout(f"user_input_queue: {user_input_queue.__dict__}\n\n", debug=True)
    
    try:
        user_input = user_input_queue.get()
        
        chat_output.append_stdout(f"USER INPUT: {user_input}\n\n")
        
        new_chat_message = create_chat_message(user_id=set_user['_id'], content=user_input, conversation_id=set_conversation['_id'])
        chat_message_id = create_model(collection_name="chat_messages", model_data=new_chat_message, client=db_client)
        chat_message_dict = get_model(collection_name="chat_messages", model_id=chat_message_id, client=db_client)
        
        chat_output.append_stdout(f"USER INPUT TASK COMPLETED\nchat message dict: {chat_message_dict}\n\n", debug=True)
        
        return chat_message_dict
    
    except Exception as e:
        # Handle the case when the queue is empty
        chat_output.append_stdout(f"EXCEPTION: {e}\n\n", debug=True)
        
        return {}

In [ ]:
def voe_task(expectation_id: ObjectId) -> dict:
    try:
        # invoke violation of expectation with LLM service
        voe_thought = mock_violation_data['voe_thought']
        
        # implement DB call
        last_llm_response = None
        
        new_violation = create_violation(last_llm_response_id=last_llm_response, expectation_id=expectation_id, voe_thought=voe_thought)
        violation_id = create_model(collection_name="violations", model_data=new_violation, client=db_client)
        violation_dict = get_model(collection_name="violations", model_id=violation_id, client=db_client)
        
        return violation_dict
    
    except Exception as e:
        chat_output.append_stdout(f"EXCEPTION: {e}\n\n", debug=True)
        
        return {}


In [ ]:
def chatbot_response_task(response_to_id: ObjectId):
    try:
        response_to_message = get_model(collection_name="chat_messages", model_id=response_to_id, client=db_client)
        
        # invoke prompt creation with useful information
        
        # invoke chatbot response with LLM Service
        
        # save final API output as "content"
        content = mock_chatbot_response_data["content"]
        
        new_chatbot_response = create_chatbot_response(content=content, conversation_id=set_conversation['_id'], response_to_id=response_to_id)
        response_id = create_model(collection_name="chatbot_responses", model_data=new_chatbot_response, client=db_client)
        response_dict = get_model(collection_name="chatbot_responses", model_id=response_id, client=db_client)
        
        return response_dict
    
    except Exception as e:
        chat_output.append_stdout(f"EXCEPTION: {e}\n\n", debug=True)

        return {}

In [ ]:
def conversation_loop():
    global text_input
    text_input = widgets.Text(placeholder='Type your message here...')
    submit_button = widgets.Button(description='Submit')
    exit_button = widgets.Button(description='Exit')

    def on_submit_button_clicked(b):
        with chat_output:
            # invoke user input task and return user message
            handle_button_click()
            message_dict = user_input_task()
            chat_output.append_stdout(f"Received message_dict: {message_dict}\n\n", debug=True)                
            message_id = message_dict["_id"]

            # invoke user prediction task and return prediction object
            expectation_dict = user_prediction_task()
            chat_output.append_stdout(f"Received prediction: {expectation_dict}\n\n", debug=True)
            expectation_id = expectation_dict["_id"]

            # invoke violation of expectation task and return violation object
            violation_dict = voe_task(expectation_id=expectation_id)
            chat_output.append_stdout(f"Received violation: {violation_dict}", debug=True)
            
            # invoke vector db fact storing with Knowledge Assessment Service
            
            # invoke chatbot response task and return chatbot message
            chatbot_response_dict = chatbot_response_task(message_id)
            chat_output.append_stdout(f"Received response: {chatbot_response_dict}", debug=True)

    submit_button.on_click(on_submit_button_clicked)

    def on_exit_button_clicked(b):
        with chat_output:
            chat_output.clear_output()
            chat_output.append_stdout("Exiting conversation loop...")

    exit_button.on_click(on_exit_button_clicked)

    display(widgets.VBox([chat_output, text_input, submit_button, exit_button]))

  

In [ ]:
def main():
    initialize_chat(saved_user_id)
    chat_output.append_stdout(f"user: {set_user}\nconversation: {set_conversation}\n\n")
    conversation_loop()

if __name__ == "__main__": main()